# **NLP2_Run1**

In [1]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 38.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.6 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [2]:

import openai
import os
from IPython.display import Markdown


### **Accessing the GPT4 API**

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']
openai.api_key = my_api_key


### **Generate Mathematical Model**

In [4]:
problem = """You are a chemist in charge of a process that requires two liquids to reach the end product. Your goal is to maximize the quality of the product in the end, which is measured by the difference between the Napierian logarithm of the product of the litres used of each component and the cube root of the summation of the first component litres as well as double the second component litres. It is required that the squared difference between the litres employed of each component and the mean litres utilised of both components is below ten. This is necessary for the obtained set to be stable. Furthermore, the mixture tank available to you can only fit 80 l. Please formulate a mathematical optimization model for this problem. """

In [5]:

client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the variables for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 1
)


In [6]:
print(response1.choices[0].message.content)

Variables:

1. L1: Litres of the first liquid
2. L2: Litres of the second liquid
3. M: Mean litres utilized of both components (calculated as (L1+L2)/2)
4. Q: The quality of the product
5. S: The squared difference between the litres employed of each component and the mean litres utilised of both components


In [7]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the objective function for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 1
)

In [8]:
print(response2.choices[0].message.content)

Objective Function:

Maximize: Q = log_e(L1*L2) - (L1 + 2*L2)^(1/3) 

(Note: "log_e" represents the Napierian logarithm, also known as the natural logarithm.)


In [9]:
response3 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the constraints for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content +  response2.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 1
)

In [10]:
print(response3.choices[0].message.content)

Constraints:

1. S = (L1 - M)^2 + (L2 - M)^2 <= 10
2. M = (L1 + L2)/2 
3. L1 + L2 <= 80
4. L1 >= 0
5. L2 >= 0
6. M >= 0
7. Q >= 0
8. S >= 0.


### **Generate Pyomo Code**

In [11]:
response4 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content + response2.choices[0].message.content + response3.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 1
)

In [12]:
Markdown(response4.choices[0].message.content)

The following python code is using the pyomo library.

The sample data used is:

    1. Max amount of liquids we can use = 80
    2. Maximum variation = 10

```python
from pyomo.environ import *

# Create a concrete model
model = ConcreteModel()

model.L1 = Var(bounds=(0, None))  # Litres of first liquid
model.L2 = Var(bounds=(0, None))  # Litres of second liquid
model.M = Var(bounds=(0, None))   # Mean litres utilized of both components
model.Q = Var(bounds=(0, None))   # Quality of product
model.S = Var(bounds=(0, None))   # Squared difference

# Objective function
def obj_expression(model):
    return model.Q == log(model.L1*model.L2) - pow((model.L1 + 2*model.L2), 1/3)
model.objective = Objective(rule=obj_expression, sense=maximize)

# Constraints
model.total_liquid = Constraint(expr= model.L1 + model.L2 <= 80)  # total amount of liquid constraint
model.mean_liquid = Constraint(expr= model.M == (model.L1 + model.L2)/2)  # mean value constraint
model.var_liquid = Constraint(expr= model.S == pow((model.L1 - model.M), 2) + pow((model.L2 - model.M), 2))  # variation constraint
model.var_liquid_bound = Constraint(expr= model.S <= 10)  # variation bound constraint

# Solve the model
solver = SolverFactory('ipopt')
solver.solve(model)
``` 

Note: Pyomo is a python-based, open-source optimization modeling language. It uses a hybrid of symbolic algebraic expressions and numerical statistical programming. Make sure to install it using pip:

    pip install pyomo

### **Run the code Generated by GPT4**

In [13]:
# Install Python API for AMPL
!pip install amplpy --upgrade

from amplpy import AMPL,ampl_notebook

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 35.1 MB/s eta 0:00:00


In [14]:
_ = load_dotenv(find_dotenv('second_knitro_license.env'))
my_knitro_license = os.environ['knitro_license']

ampl = ampl_notebook(
    modules=["knitro"],  # modules to install
    license_uuid=my_knitro_license,  # license to use
)

Licensed to AMPL Community Edition License for <cevikmelis14@gmail.com>.


In [16]:
from pyomo.environ import *

# Create a concrete model
model = ConcreteModel()

model.L1 = Var(bounds=(0, None))  # Litres of first liquid
model.L2 = Var(bounds=(0, None))  # Litres of second liquid
model.M = Var(bounds=(0, None))   # Mean litres utilized of both components
model.Q = Var(bounds=(0, None))   # Quality of product
model.S = Var(bounds=(0, None))   # Squared difference

# Objective function
def obj_expression(model):
    return model.Q == log(model.L1*model.L2) - pow((model.L1 + 2*model.L2), 1/3)
model.objective = Objective(rule=obj_expression, sense=maximize)

# Constraints
model.total_liquid = Constraint(expr= model.L1 + model.L2 <= 80)  # total amount of liquid constraint
model.mean_liquid = Constraint(expr= model.M == (model.L1 + model.L2)/2)  # mean value constraint
model.var_liquid = Constraint(expr= model.S == pow((model.L1 - model.M), 2) + pow((model.L2 - model.M), 2))  # variation constraint
model.var_liquid_bound = Constraint(expr= model.S <= 10)  # variation bound constraint

# Solve the model
solver = SolverFactory('knitro')
solver.solve(model)

ERROR:pyomo.core:Rule failed when generating expression for Objective objective with index None:
ValueError: Cannot assign EqualityExpression to 'objective': ScalarObjective components only allow numeric expression types.
ERROR:pyomo.core:Constructing component 'objective' from data=None failed:
    ValueError: Cannot assign EqualityExpression to 'objective': ScalarObjective components only allow numeric expression types.


ValueError: Cannot assign EqualityExpression to 'objective': ScalarObjective components only allow numeric expression types.

In [21]:
from pyomo.environ import *

# Create a concrete model
model = ConcreteModel()

model.L1 = Var(bounds=(0, None))  # Litres of first liquid
model.L2 = Var(bounds=(0, None))  # Litres of second liquid
model.M = Var(bounds=(0, None))   # Mean litres utilized of both components
model.Q = Var(bounds=(0, None))   # Quality of product
model.S = Var(bounds=(0, None))   # Squared difference

# Objective function
def obj_expression(model):
    return log(model.L1*model.L2) - pow(model.L1 + 2*model.L2,1/3)
model.objective = Objective(rule=obj_expression, sense=maximize)

# Constraints
model.total_liquid = Constraint(expr= model.L1 + model.L2 <= 80)  # total amount of liquid constraint
model.mean_liquid = Constraint(expr= model.M == (model.L1 + model.L2)/2)  # mean value constraint
model.var_liquid = Constraint(expr= model.S == pow((model.L1 - model.M), 2) + pow((model.L2 - model.M), 2))  # variation constraint
model.var_liquid_bound = Constraint(expr= model.S <= 10)  # variation bound constraint

# Solve the model
solver = SolverFactory('knitro')
solver.solve(model)
print(model.objective())
print(model.L1())
print(model.L2())

2.4730339021209895
42.236065785547574
37.76393018195405


### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**